In [31]:

import requests,os,time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(0.5) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [32]:
connector = Connector('Cleaning')
from bs4 import BeautifulSoup
import seaborn as sns
import pandas as pd
import numpy as np
import requests

In [33]:
# SAMLET
gross=[]
rating=[]
movie_stars=[]
director=[]
year=[]
name=[]
genre=[]
summary=[]

for v in range (1,201):
    url = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page={}&title_type=movie&fbclid=IwAR3B7G9VdhKWjVvbFIPhdH9vGLZwmO_zzTeNlCj4whUMbn_RtS3g1g9FiUQ&release_date=1980%2C2019&sort=num_votes,desc'.format(v)
    response,callid = connector.get(url,'Exam')
    html = response.text
    soup = BeautifulSoup(html,'lxml')
    for j in range(50):
        box=soup.findAll('div',{"class":'lister-item mode-detail'})[j]
# BOX OFFICE
        if len(box.findAll("span", {"name": "nv"})) < 2:
            gross.append(None)
        if len(box.findAll("span", {"name": "nv"})) == 2:
            gross.append(box.findAll("span", {"name": "nv"})[1].text)
# Rating
        if box.findAll("div", {"class": "inline-block ratings-imdb-rating"})==[]:
            rating.append(None)
        else:
            primo=box.findAll("div", {"class": "inline-block ratings-imdb-rating"})[0].text
            rating.append(primo.split('\n')[2])
# Movie Stars
        if box.findAll("p", {"class":"text-muted text-small"})== []:
            movie_stars.append('NaN')
        else: 
            stars = box.findAll("p", {"class":"text-muted text-small"})[1].text.strip()
            if len(stars.split(':'))<3:
                movie_stars.append('NaN')
            else:
                stars_1 = stars.split(':')[2].split('\n')[1:]
                movie_stars.append(stars_1)
# Directors
        di=box.findAll("p", {"class": "text-muted text-small"})[1].text
        if len(di.split('Stars')[0].split('\n')) > 5:
            director.append(di.split('Stars')[0].split('\n')[2:4])#+primo.split('Stars')[0].split('\n')[3])
        else:
            director.append(di.split('Stars')[0].split('\n')[2])
# Years
        headline_j = soup.findAll('h3')[j] # search for the first headline: h1 tag. 
        #name = headline_i['class'][0].strip() # use the class attribute name as column name.
        value = headline_j.text.strip() # extract text using build in method        
        film = value.split('\n')[2]
        if len(film)> 6:
            film = value.split(' ')[-1]
        year.append(film[1:5])
# Name
        headline_j = soup.findAll('h3')[j] # search for the first headline: h1 tag. 
        #name = headline_i['class'][0].strip() # use the class attribute name as column name.
        value = headline_j.text.strip() # extract text using build in method.
        film = value.split('\n')[1]
        name.append(film)
# Genre
        if box.findAll("span", {"class":"genre"}) ==[]:
            genre.append(None)
        else: 
            movie_genre = box.findAll("span", {"class":"genre"})[0].text.strip()
            genre.append(movie_genre)
# Summary
        if box.findAll("p", {"class":""})== []:
            summary.append(None)
        else: 
            movie_summary = box.findAll("p", {"class":""})[0].text.strip()
            summary.append(movie_summary)

In [8]:
Star_1_fn = []

for i in range (10000): 
    Star_1 = movie_stars[i][0].strip().split(',')[0].split(' ')[0]
    Star_1_fn.append(Star_1)  

#for i in range (10000): 
#    Star_2 = movie_stars[i][1].strip().split(',')[0].split(' ')[0]
#    Star_2_fn.append(Star_2)
    
#for i in range (10000): 
#    Star_3 = movie_stars[i][2].strip().split(',')[0].split(' ')[0]
#    Star_3_fn.append(Star_3)
    
#for i in range (10000): 
#    Star_4 = movie_stars[i][3].strip().split(',')[0].split(' ')[0]
#    Star_4_fn.append(Star_4)



IndexError: list index out of range

In [27]:
# SCRABE FEMALE FIRST NAMES AND OTHERS
first_female=[]
sur_female=[]
for i in range(1,8):
    url='https://imdb.com/list/ls022928836/?sort=list_order,asc&mode=detail&page={}'.format(i)
    call='Exam, KU, Female Page {}'.format(i)
    response,call_id = connector.get(url, call)
    html = response.text
    soup=str(html)
    if i < 7:
        for i in range(1,101):
            first_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))
    if i == 7:
        for i in range(1,7):
            first_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))


In [28]:
first_female[:5]

['Natalie', 'Anne', 'Talia', 'Diane', 'Keira']

In [29]:
# SCRABE MALE FIRST NAMES AND OTHERS
first_male=[]
sur_male=[]
for i in range (1,8):
    url='https://www.imdb.com/list/ls022928819/?sort=list_order,asc&mode=detail&page={}'.format(i)
    call='Exam, KU, Male Page {}'.format(i)
    response,call_id = connector.get(url, call)
    html = response.text
    soup=str(html)
    if i < 7:
        for i in range(1,101):
            first_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))
    if i == 7:
        for i in range(1,82):
            first_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))


In [30]:
first_male[:5]

['Morgan', 'Leonardo', 'Brad', 'Michael', 'Robert']

In [21]:
import nltk
name=nltk.corpus.names
# CHECKING FOR OVERLAP IN THE ORIGINAL LIST
name.fileids()
name.words('male.txt') in name.words('female.txt')
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3
check=intersection(first_female, name.words('female.txt'))
# REMOVING DUPLICATES BY MAKING INTO DICT AND BACK AGAIN
first_m=list(dict.fromkeys(first_male))
first_f=list(dict.fromkeys(first_female))

male_name=name.words('male.txt')+first_m
print(len(male_name))
first_m_done=list(dict.fromkeys(male_name))
print(len(first_m_done))
print(len(name.words('male.txt')))

female_name=name.words('female.txt')+first_f
print(len(female_name))
first_f_done=list(dict.fromkeys(female_name))
print(len(first_f_done))
print(len(name.words('female.txt')))

3329
3049
2943
5425
5127
5001


In [24]:
all_female = first_f_done
all_male = first_m_done


In [26]:
print(all_female,  file=open('all_female.txt', 'w'))
print(all_male, file = open('all_male.txt','w'))
print(Star_1_fn, file = open('Top Star first name', 'w')